In [25]:
import pandas as pd
import numpy as np
from pyomo.environ import *
from math import sqrt
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition

### Modelo Original do TSP

O modelo original do TSP que vamos implementar baseia-se na formulação de Miller-Tucker-Zemlin (MTZ) apresentada. Este modelo usa variáveis binárias para indicar se uma aresta (i, j) está no ciclo, e variáveis de posicionamento para evitar subcircuitos.

### Modelo Fortalecido do TSP

O modelo fortalecido do TSP inclui as desigualdades de Miller-Tucker-Zemlin fortalecidas, que visam aproximar mais a formulação da envoltória convexa das soluções viáveis do problema, potencialmente melhorando o desempenho do algoritmo de Branch-and-Bound.

### Leitura e Preparação dos Dados


In [16]:

data = {
    'Produtor': range(1, 22),
    'Leste': [0, -3, 1, 4, -5, -5, -4, 6, 3, -1, 0, 6, 2, -2, 6, 1, -3, -6, 2, -6, 5],
    'Norte': [0, 3, 11, 7, 9, -2, -7, 0, -6, -3, -6, 4, 5, 8, 10, 8, 1, 5, 9, -5, -4]
}

locations = pd.DataFrame(data)
locations['Leste'] = locations['Leste'] * 10
locations['Norte'] = locations['Norte'] * 10

# Distância Euclidiana
def calc_distance(row1, row2):
    return sqrt((row1['Leste'] - row2['Leste'])**2 + (row1['Norte'] - row2['Norte'])**2)

# Matriz de distância
distance_matrix = np.zeros((len(locations), len(locations)))
for i in range(len(locations)):
    for j in range(len(locations)):
        distance_matrix[i][j] = calc_distance(locations.iloc[i], locations.iloc[j])

# Lista de vertices (produtores incluindo o depósito)
V = list(range(len(locations)))


#### Modelo Base do TSP (MTZ)

In [17]:
def create_tsp_model(V, distance_matrix):
    model = ConcreteModel()
    
    # Conjuntos
    model.V = Set(initialize=V)
    model.A = Set(within=model.V*model.V, initialize=[(i,j) for i in V for j in V if i!=j])
    
    # Parâmetros
    model.d = Param(model.A, initialize={(i,j): distance_matrix[i][j] for i,j in model.A})
    
    # Variáveis de decisão
    model.x = Var(model.A, within=Binary)
    model.u = Var(model.V, within=NonNegativeReals)
    
    # Função objetivo
    model.obj = Objective(expr=sum(model.d[i,j] * model.x[i,j] for i,j in model.A), sense=minimize)
    
    # Restrições
    model.degree_out = Constraint(model.V, rule=lambda m, i: sum(m.x[i,j] for j in m.V if (i,j) in m.A) == 1)
    model.degree_in = Constraint(model.V, rule=lambda m, i: sum(m.x[j,i] for j in m.V if (j,i) in m.A) == 1)
    model.subtour_elimination = Constraint(model.A, rule=lambda m, i, j: m.u[i] + 1 <= m.u[j] + len(V)*(1 - m.x[i,j]) if i != 0 and j != 0 else Constraint.Skip)
    model.u[0].fix(0)
    
    return model

### Modelo Fortalecido do TSP (SMTZ)


In [18]:
def create_tsp_strong_model(V, distance_matrix):
    model = ConcreteModel()
    
    # Conjuntos
    model.V = Set(initialize=V)
    model.A = Set(within=model.V*model.V, initialize=[(i,j) for i in V for j in V if i!=j])
    
    # Parâmetros
    model.d = Param(model.A, initialize={(i,j): distance_matrix[i][j] for i,j in model.A})
    
    # Variáveis de decisão
    model.x = Var(model.A, within=Binary)
    model.u = Var(model.V, within=NonNegativeReals)
    
    # Função objetivo
    model.obj = Objective(expr=sum(model.d[i,j] * model.x[i,j] for i,j in model.A), sense=minimize)
    
    # Restrições
    model.degree_out = Constraint(model.V, rule=lambda m, i: sum(m.x[i,j] for j in m.V if (i,j) in m.A) == 1)
    model.degree_in = Constraint(model.V, rule=lambda m, i: sum(m.x[j,i] for j in m.V if (j,i) in m.A) == 1)
    model.subtour_elimination_strong = Constraint(model.A, rule=lambda m, i, j: m.u[i] + (len(V) - 1)*m.x[i,j] + (len(V) - 3)*m.x[j,i] - (len(V) - 2) <= m.u[j] if i != 0 and j != 0 else Constraint.Skip)
    model.u[0].fix(0)
    
    return model



### Execução dos Modelos e Comparação dos Resultados

Agora, implementaremos a execução desses modelos usando o solver 'gurobi'.



In [22]:
def solve_tsp(model):
    solver = SolverFactory('gurobi')    
    result = solver.solve(model, tee=True)    
    
    if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
        print("Solution is optimal")
    elif result.solver.termination_condition == TerminationCondition.infeasible:
        print("No feasible solution found")
    else:
        print("Solver Status: ", result.solver.status)
        
    print("Objective Value: ", model.obj())
    print("Gurobi Solver Time: ", result.solver.time, "seconds")
    for i,j in model.A:
        if model.x[i,j].value > 0.5:
            print(f"Edge from {i} to {j} is in the solution")
    print("\n")

model_original = create_tsp_model(V, distance_matrix)
model_strong = create_tsp_strong_model(V, distance_matrix)

In [23]:
print("Solving Original TSP Model")
solve_tsp(model_original)

Solving Original TSP Model


Set parameter CSManager to value "http://110.22.10.6:61080"
Set parameter CSAuthToken
Compute Server job ID: 43e74368-f454-4dd4-8791-3d944573af83
Capacity available on 'SRVAIOPT01' - connecting...
Established HTTP unencrypted connection
Read LP format model from file C:\Users\JEANLU~1.MOT\AppData\Local\Temp\tmpbgq5u8_n.pyomo.lp
Reading time = 0.48 seconds
x1: 422 rows, 440 columns, 1980 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Gurobi Compute Server Worker version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 422 rows, 440 columns and 1980 nonzeros
Model fingerprint: 0xe825face
Variable types: 20 continuous, 420 integer (420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.00s
Presolved: 422 rows, 440 columns, 1980 nonzeros
Variable types: 20 conti

In [24]:
print("Solving Strong TSP Model")
solve_tsp(model_strong)

Solving Strong TSP Model


Set parameter CSManager to value "http://110.22.10.6:61080"
Set parameter CSAuthToken
Compute Server job ID: f53ac5e6-5865-4649-9d5e-1db8b9bcb9a9
Capacity available on 'SRVAIOPT01' - connecting...
Established HTTP unencrypted connection
Read LP format model from file C:\Users\JEANLU~1.MOT\AppData\Local\Temp\tmp7vjj6c3o.pyomo.lp
Reading time = 0.50 seconds
x1: 422 rows, 440 columns, 2360 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Gurobi Compute Server Worker version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 422 rows, 440 columns and 2360 nonzeros
Model fingerprint: 0xa3507a9a
Variable types: 20 continuous, 420 integer (420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.00s
Presolved: 422 rows, 440 columns, 2360 nonzeros
Variable types: 20 conti

### Comparação dos Resultados
Ambos os modelos chegaram à mesma solução ótima com o valor da função objetivo sendo **697.1032374739405**. No entanto, existem diferenças significativas no processo de otimização que precisam ser destacadas.

### Desempenho do Algoritmo Branch-and-Bound

1. **Modelo Original do TSP:**
   - **Tempo de execução:** Aproximadamente 9.59 segundos.
   - **Exploração de Nós:** 1 nó foi explorado, com 390 iterações do simplex.
   - **Geração de cortes:** Foram aplicados cortes de Gomory e MIR.
   - **Variações na Objetivo:** Inicialmente sem solução, encontrou 1256.26, reduziu para 730.321 e finalmente otimizou para 697.103.
   - **Gap de solução:** O gap inicial foi de 45.5%, reduzido para 0.74%, e finalmente fechado a 0.00%.

2. **Modelo Fortalecido do TSP (SMTZ):**
   - **Tempo de execução:** Aproximadamente 9.89 segundos.
   - **Exploração de Nós:** 1 nó foi explorado, com 140 iterações do simplex.
   - **Geração de cortes:** Foram aplicados cortes de Gomory e MIR.
   - **Variações na Objetivo:** Inicialmente sem solução, encontrou 821.381, reduziu para 697.103 e não houve mais mudanças significativas.
   - **Gap de solução:** O gap inicial foi de 15.8%, rapidamente reduzido para 0.27% e então fechado a 0.00%.

### Análise dos Resultados

- **Eficiência na Relaxação Linear:** O modelo fortalecido começou com uma solução de relaxação linear mais apertada (691.92769) do que o modelo original (684.81383). Isso indica que a relaxação linear do modelo fortalecido é mais próxima do conjunto convexo das soluções inteiras.
- **Rapidez na Convergência:** Embora o tempo de execução total seja similar, o modelo fortalecido apresentou uma convergência mais rápida para a solução ótima, com menos iterações do simplex (140 vs. 390). Isso sugere que o modelo fortalecido pode ser mais eficiente em problemas maiores ou mais complexos.
- **Estabilidade e Robustez:** O modelo fortalecido mostrou-se mais estável em termos de gap de otimização, reduzindo rapidamente o gap e mantendo uma consistência na aproximação da solução ótima.

### Conclusão

A incorporação das desigualdades fortalecidas de Miller-Tucker-Zemlin no modelo fortalecido parece ter melhorado a qualidade da relaxação linear e diminuído o número de iterações necessárias para alcançar a solução ótima, o que pode ser um indicativo de melhor desempenho em cenários mais exigentes. Embora o tempo total de execução não tenha apresentado diferenças significativas, a eficiência na exploração dos nós e a rapidez na convergência são vantagens claras do modelo fortalecido.

Portanto, para problemas maiores ou mais complexos de TSP, a adoção do modelo fortalecido pode ser mais benéfica, especialmente em termos de estabilidade e eficiência computacional.